In [1]:
from dotenv import load_dotenv
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI  # 或其他LLM
from langchain.memory import ConversationBufferWindowMemory
from typing import List, Dict, Any
import json

from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import os


# 1. 定义商品获取工具
class ProductSearchTool:
    """模拟商品搜索工具"""

    @tool
    def search_products(self, query: str) -> List[Dict[str, Any]]:
        """
        模拟根据查询条件获取商品列表
        实际应用中这里可以连接数据库或调用API
        """
        # 模拟商品数据库
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        # 简单模拟搜索逻辑 - 实际应用中可以更复杂
        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [
    Tool(
        name="Product Search",
        func=product_tool.search_products,
        description="""用于搜索商品。输入应该是用户的商品需求描述，
        如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"""
    )
]

# 3. 定义提示词模板
template = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

用户当前需求: {input}

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

prompt = PromptTemplate(
    input_variables=["input"],
    template=template
)

# 4. 设置记忆和LLM
# memory = ConversationBufferWindowMemory(k=3)  # 记住最近3轮对话
# llm = OpenAI(temperature=0.3)  # 控制创造性，商品推荐需要准确性

# 5. 创建LLM链
# llm_chain = LLMChain(llm=llm, prompt=prompt)

# 6. 定义自定义输出解析器
class ProductOutputParser:
    def parse(self, text: str):
        # 这里可以添加更复杂的解析逻辑
        return text.strip()

output_parser = ProductOutputParser()

# 7. 智能体创建
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE")
)
llm_with_tools = llm.bind_tools(tools=tools)
llm = prompt|llm_with_tools|StrOutputParser()

# 9. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        # for res in llm.stream({"input": user_input}):
        #     print(res)
        response = llm.invoke(input=user_input,stream=True)
        # response = llm.stream({"input":user_input})
        # print(f"tool_calls: {response.tool_calls}")
        print(f"商品推荐助手: {response}")

        break
if __name__ == "__main__":
    run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)


KeyboardInterrupt: Interrupted by user

In [25]:
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType, Tool
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from typing import List, Dict, Any
import os

# 1. 定义商品获取工具
class ProductSearchTool:
    """模拟商品搜索工具"""

    @tool
    def search_products(self, query: str) -> List[Dict[str, Any]]:
        """模拟根据查询条件获取商品列表"""
        # 模拟商品数据库
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        # 简单搜索逻辑
        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [
    Tool(
        name="Product Search",
        func=product_tool.search_products,
        description="""用于搜索商品。输入应该是用户的商品需求描述，
        如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"""
    )
]

# 3. 定义提示词模板 - 使用新的ChatPromptTemplate
system_message = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

human_message = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    # MessagesPlaceholder(variable_name="history"),
    ("human", human_message)
])

# 4. 设置记忆
# memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 5. 加载环境变量并初始化LLM
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
)

# 6. 创建Agent - 使用新的函数调用风格
agent = {
    "input": lambda x: x["input"],
    # "history": lambda x: x["history"]
} | prompt | llm.bind(
    functions=[
        {
            "name": "search_products",
            "parameters": {
                "query": {"type": "string", "description": "用户的商品需求描述"}
            },
            "description": "搜索匹配的商品"
        }
    ]
)

# 7. 定义工具执行逻辑
def execute_function_call(output):
    if output.get("function_call"):
        function_call = output["function_call"]
        name = function_call["name"]
        parameters = function_call["parameters"]

        if name == "search_products":
            tool_result = product_tool.search_products(parameters["query"])
            return {
                "name": name,
                "parameters": parameters,
                "output": tool_result
            }
    return None

# 8. 创建Agent执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    # memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# 9. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.invoke({"input": user_input})
        print(f"商品推荐助手: {response['output']}")

if __name__ == "__main__":
    run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)


> Entering new AgentExecutor chain...


Error in StdOutCallbackHandler.on_agent_action callback: AttributeError("'tuple' object has no attribute 'log'")


AttributeError: 'tuple' object has no attribute 'tool'

In [27]:
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType, Tool
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from typing import List, Dict, Any
import os

# 1. 定义商品获取工具
class ProductSearchTool:
    """模拟商品搜索工具"""
    @tool
    def search_products(self, query: str) -> List[Dict[str, Any]]:
        """模拟根据查询条件获取商品列表"""
        # 模拟商品数据库
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        # 简单搜索逻辑
        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [
    Tool(
        name="Product Search",
        func=product_tool.search_products,
        description="""用于搜索商品。输入应该是用户的商品需求描述，
        如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"""
    )
]

# 3. 定义提示词模板 - 使用新的ChatPromptTemplate
system_message = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

human_message = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="history"),
    ("human", human_message)
])

# 4. 设置记忆
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 5. 加载环境变量并初始化LLM
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 使用有效的OpenAI模型
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
)

# 6. 创建Agent - 使用新的函数调用风格
agent = {
    "input": lambda x: x["input"],
    "history": lambda x: x["history"]
} | prompt | llm.bind(
    functions=[
        {
            "name": "search_products",
            "parameters": {
                "query": {"type": "string", "description": "用户的商品需求描述"}
            },
            "description": "搜索匹配的商品"
        }
    ]
)

# 7. 定义工具执行逻辑
def execute_function_call(output):
    if output.get("function_call"):
        function_call = output["function_call"]
        name = function_call["name"]
        parameters = function_call["parameters"]

        if name == "search_products":
            tool_result = product_tool.search_products(parameters["query"])
            return {
                "name": name,
                "parameters": parameters,
                "output": tool_result
            }
    return None

# 8. 创建Agent执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

# 9. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.invoke({"input": user_input})
        print(f"商品推荐助手: {response['output']}")

if __name__ == "__main__":
    run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)


> Entering new AgentExecutor chain...


Error in StdOutCallbackHandler.on_agent_action callback: AttributeError("'tuple' object has no attribute 'log'")


AttributeError: 'tuple' object has no attribute 'tool'

In [29]:
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType, initialize_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from typing import List, Dict, Any
import os

# 1. 定义商品获取工具
class ProductSearchTool:
    """模拟商品搜索工具"""

    @tool
    def search_products(self, query: str) -> List[Dict[str, Any]]:
        """模拟根据查询条件获取商品列表"""
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [
    {
        "name": "Product Search",
        "func": product_tool.search_products,
        "description": "用于搜索商品。输入应该是用户的商品需求描述，如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "用户的商品需求描述"
                }
            },
            "required": ["query"]
        }
    }
]

# 3. 定义提示词模板 - 使用新的ChatPromptTemplate
system_message = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

human_message = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="history"),
    ("human", human_message)
])

# 4. 设置记忆
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 5. 加载环境变量并初始化LLM
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 使用有效的OpenAI模型
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
)

# 6. 初始化Agent（关键修正点：指定Agent类型为OPENAI_FUNCTIONS）
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent.run(user_input)  # 使用run方法简化调用
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    run_conversation()

TypeError: Can't instantiate abstract class BaseTool without an implementation for abstract method '_run'

In [30]:
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType, Tool
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from typing import List, Dict, Any
import os

# 1. 定义商品获取工具（继承BaseTool）
class ProductSearchTool(BaseTool):
    """模拟商品搜索工具"""
    name = "Product Search"
    description = "用于搜索商品。输入应该是用户的商品需求描述，如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        """实现BaseTool要求的_run方法"""
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        """异步方法实现（可选）"""
        return self._run(query)

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [product_tool]  # 直接使用工具实例

# 3. 定义提示词模板 - 使用新的ChatPromptTemplate
system_message = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

human_message = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="history"),
    ("human", human_message)
])

# 4. 设置记忆
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 5. 加载环境变量并初始化LLM
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 使用有效的OpenAI模型
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
)

# 6. 初始化Agent（关键修正点：指定Agent类型为OPENAI_FUNCTIONS）
agent = AgentExecutor.from_llm_and_tools(
    llm=llm,
    tools=tools,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent.run(user_input)  # 使用run方法简化调用
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    run_conversation()

PydanticUserError: Field 'name' defined on a base class was overridden by a non-annotated attribute. All field definitions, including overrides, require a type annotation.

For further information visit https://errors.pydantic.dev/2.11/u/model-field-overridden

In [33]:
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, AgentType
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from typing import List, Dict, Any
import os

# 1. 定义商品获取工具（添加类型注解）
class ProductSearchTool(BaseTool):
    """模拟商品搜索工具"""
    name: str = "Product Search"  # 添加类型注解
    description: str = "用于搜索商品。输入应该是用户的商品需求描述，如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"  # 添加类型注解

    def _run(self, query: str) -> List[Dict[str, Any]]:
        """实现BaseTool要求的_run方法"""
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        """异步方法实现（可选）"""
        return self._run(query)

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [product_tool]  # 直接使用工具实例

# 3. 定义提示词模板
system_message = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出

请开始完成任务:"""

human_message = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="history"),
    ("human", human_message)
])

# 4. 设置记忆
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 5. 加载环境变量并初始化LLM
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 使用有效的OpenAI模型
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
)

# 6. 初始化Agent
agent = AgentExecutor.from_llm_and_tools(
    llm=llm,
    tools=tools,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent.run(user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    run_conversation()

AttributeError: from_llm_and_tools

In [37]:
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from typing import List, Dict, Any
import os

# 1. 定义商品获取工具
class ProductSearchTool(BaseTool):
    """模拟商品搜索工具"""
    name: str = "Product Search"
    description: str = "用于搜索商品。输入应该是用户的商品需求描述，如'蓝牙耳机'或'智能家居设备'。输出是匹配的商品列表。"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        """实现BaseTool要求的_run方法"""
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMaster"},
            {"id": 2, "name": "智能手表", "price": 999, "category": "电子产品", "brand": "TechGiant"},
            {"id": 3, "name": "有机棉T恤", "price": 89, "category": "服装", "brand": "EcoWear"},
            {"id": 4, "name": "不锈钢保温杯", "price": 129, "category": "生活用品", "brand": "KeepHot"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerUp"},
        ]

        query = query.lower()
        results = []
        for product in product_db:
            if (query in product["name"].lower() or
                    query in product["category"].lower() or
                    query in product["brand"].lower()):
                results.append(product)

        return results[:3]  # 返回最多3个结果

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        """异步方法实现（可选）"""
        return self._run(query)

# 2. 创建工具实例
product_tool = ProductSearchTool()

tools = [product_tool]

# 3. 定义提示词模板
system_message = """你是一个智能商品推荐助手，帮助用户找到他们需要的商品。
你可以访问商品数据库来查找匹配的商品。

你应该:
1. 分析用户需求，理解他们想要什么类型的商品
2. 必须使用工具查找匹配的商品
3. 向用户推荐最合适的1-3个商品，说明推荐理由
4. 如果找不到匹配商品，请明确输出提示信息，礼貌地告知用户并尝试询问更详细的需求
5. 如果你有调用工具进行查询，请在回应中显式地指出使用了工具，并指出工具的返回结果

请开始完成任务:"""

human_message = "{input}"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    MessagesPlaceholder(variable_name="history"),
    ("human", human_message)
])

# 4. 设置记忆
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 5. 加载环境变量并初始化LLM
load_dotenv()
llm = ChatOpenAI(
    model="deepseek-ai/DeepSeek-R1",  # 使用有效的OpenAI模型
    temperature=0,
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_API_BASE", "https://api.openai.com/v1")
)

# 6. 初始化Agent（使用initialize_agent函数）
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    memory=memory,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent.run(input=user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)


> Entering new AgentExecutor chain...

关于**蓝牙耳机**，以下是您可能需要了解的关键信息，涵盖选购要点、技术参数、常见问题及使用技巧：

---

### **一、核心选购指南**
1. **类型选择**：
   - **真无线（TWS）**：如 AirPods Pro、Galaxy Buds（便携性强，易丢失）
   - **颈挂式**：如 Sony WI-1000XM（续航长，不易丢）
   - **头戴式**：如 Bose 700（音质/降噪更优，体积大）

2. **关键参数**：
   - **蓝牙版本**：选 5.0+（传输更稳、功耗更低）
   - **编解码器**：优先 LDAC/AptX HD（高音质）＞ AAC/SBC（基础）
   - **续航时间**：单次 4h+，总续航 20h+ 为佳
   - **防水等级**：IPX4（防汗）~ IPX7（可浸水）

3. **功能需求**：
   - **主动降噪（ANC）**：通勤/办公首选（索尼、Bose 表现佳）
   - **通透模式**：监听环境音（安全场景必备）
   - **多设备连接**：需同时连手机/电脑选此功能
   - **低延迟模式**：游戏/视频用户重点看（<100ms）

---

### **二、主流品牌对比**
| **品牌** | **优势**                  | **代表型号**       |
|----------|--------------------------|-------------------|
| **索尼** | 降噪顶尖，音质均衡        | WF-1000XM5        |
| **Bose** | 舒适度+降噪标杆           | QuietComfort Earbuds II |
| **苹果** | 生态无缝联动              | AirPods Pro 2     |
| **三星** | 安卓适配优，性价比高      | Galaxy Buds2 Pro  |
| **华为** | 多设备协同，骨声纹识别    | FreeBuds Pro 3    |

---



KeyboardInterrupt: 

In [38]:
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI  # 注意：这里使用OpenAI类但配置DeepSeek的API
from langchain.tools import BaseTool
from typing import List, Dict, Any
import os
import requests
import json

# 1. 定义商品搜索工具
class ProductSearchTool(BaseTool):
    name: str = "Product Search"
    description: str = "搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        """实际应用中可连接真实的商品数据库或API"""
        # 模拟搜索结果
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
            {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
            {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
            {"id": 4, "name": "机械键盘", "price": 499, "category": "电子产品", "brand": "KeyMaster", "features": "RGB背光"},
            {"id": 5, "name": "便携式充电宝", "price": 159, "category": "电子产品", "brand": "PowerPlus", "features": "20000mAh容量"},
        ]

        # 简单匹配逻辑
        results = []
        for product in product_db:
            if query.lower() in product["name"].lower() or query.lower() in product["category"].lower():
                results.append(product)

        return results[:3]  # 返回最多3个结果

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        """异步版本（可选）"""
        return self._run(query)

# 2. 定义硅基流动DeepSeek模型包装器
class DeepSeekLLM(OpenAI):
    """
    适配硅基流动平台的DeepSeek模型
    注意：这里继承OpenAI类是为了兼容LangChain接口，实际调用的是硅基流动的DeepSeek API
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # 从环境变量获取硅基流动API配置
        self.siliconflow_api_key = os.environ.get("SILICONFLOW_API_KEY")
        self.siliconflow_api_base = os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
        self.model_name = kwargs.get("model_name", "deepseek-ai/deepseek-chat")

    def _call(self, prompt: str, stop=None) -> str:
        """重写调用方法，指向硅基流动的DeepSeek API"""
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }

        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": self.max_tokens
        }

        # 调用硅基流动API
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )

        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")

        return response.json()["choices"][0]["message"]["content"]

# 3. 初始化工具和模型
load_dotenv()  # 从.env文件加载环境变量

tools = [ProductSearchTool()]

# 初始化DeepSeek模型（通过硅基流动平台）
llm = DeepSeekLLM(
    model_name="deepseek-ai/deepseek-chat",  # 选择合适的DeepSeek模型
    temperature=0.2,  # 控制输出随机性
    openai_api_key=os.environ.get("OPENAI_API_API_KEY"),  # 实际使用硅基流动的API密钥
    openai_api_base=os.environ.get("OPENAI_API_API_BASE", "https://api.siliconflow.cn/v1")
)

# 4. 配置并初始化Agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # 使用REACT框架，适合非OpenAI模型
    verbose=True,
    handle_parsing_errors=True
)

# 5. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent.run(user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    # 确保设置了环境变量
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()

ValueError: "DeepSeekLLM" object has no field "siliconflow_api_key"

In [40]:
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.schema import LLMResult
from langchain.llms.base import LLM
from langchain.tools import BaseTool
from typing import List, Dict, Any, Optional
import os
import requests
import json

# 1. 定义商品搜索工具
class ProductSearchTool(BaseTool):
    name: str = "Product Search"
    description: str = "搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        """模拟搜索结果"""
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
            {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
            {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
        ]

        results = [p for p in product_db if query.lower() in p["name"].lower()]
        return results[:3]

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        return self._run(query)

# 2. 自定义DeepSeek LLM类（不继承OpenAI）
class DeepSeekLLM(LLM):
    """兼容LangChain的DeepSeek模型包装器"""
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """调用硅基流动的DeepSeek API"""
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }

        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }

        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )

        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")

        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        """生成多个提示的响应"""
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            results.append({"text": text})
        return LLMResult(generations=results)

# 3. 初始化工具和模型
load_dotenv()

tools = [ProductSearchTool()]

llm = DeepSeekLLM(
    model_name="deepseek-ai/DeepSeek-R1",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 4. 配置并初始化Agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

# 5. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent.run(user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)


> Entering new AgentExecutor chain...


ValidationError: 1 validation error for LLMResult
generations.0
  Input should be a valid list [type=list_type, input_value={'text': '\nThought: 用...一步缩小范围！'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [41]:
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent
from langchain.schema import LLMResult
from langchain.llms.base import LLM
from langchain.tools import BaseTool
from typing import List, Dict, Any, Optional
import os
import requests
import json
import re

# 1. 定义商品搜索工具
class ProductSearchTool(BaseTool):
    name: str = "Product Search"
    description: str = "搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
            {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
            {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
        ]
        results = [p for p in product_db if query.lower() in p["name"].lower()]
        return results[:3]

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        return self._run(query)

# 2. 自定义DeepSeek LLM类（含输出解析）
class DeepSeekLLM(LLM):
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            # 新增：解析模型输出中的实际回复内容
            parsed_text = self._parse_llm_output(text)
            results.append({"text": parsed_text})
        return LLMResult(generations=results)

    def _parse_llm_output(self, text: str) -> str:
        """解析DeepSeek模型输出，提取实际回复内容"""
        # 示例：移除Thought: 等思考过程（可根据实际输出调整）
        match = re.search(r"Thought: (.*?)\n\n(.*)", text, re.DOTALL)
        if match:
            return match.group(2).strip()
        return text.strip()

# 3. 初始化工具和模型
load_dotenv()

tools = [ProductSearchTool()]

llm = DeepSeekLLM(
    model_name="deepseek-ai/deepseek-chat",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 4. 配置并初始化Agent（显式设置输出解析器）
from langchain.agents import Tool, ZeroShotAgent
from langchain.prompts import PromptTemplate

# 定义工具描述
tool_descriptions = """
Product Search: 用于搜索商品数据库。输入应为商品需求描述，如'蓝牙耳机'或'运动手表'，输出为匹配的商品列表。
"""

# 创建提示词模板
prefix = """
你是商品推荐助手，必须遵循以下规则：
1. 所有推荐必须先调用Product Search工具
2. 工具调用格式：<|FunctionCallBegin|>[{"name":"Product Search","parameters":{"query":"需求描述"}}]<|FunctionCallEnd|>
3. 若工具返回结果，需按格式推荐并说明理由
4. 若无结果，需询问用户更详细的需求
"""

suffix = """
问题: {input}
{intermediate_steps}
思考: """

prompt = PromptTemplate(
    input_variables=["input", "intermediate_steps"],
    output_parser=ZeroShotAgent.get_default_output_parser(),
    template=prefix + suffix,
)

# 创建Agent
agent = ZeroShotAgent(llm_chain=llm, tools=tools, prompt=prompt, verbose=True)
agent_executor = initialize_agent(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 5. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        # 关键修改：使用agent_executor.run替代agent.run
        response = agent_executor.run(user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()

AttributeError: get_default_output_parser

In [42]:
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent, Tool, ZeroShotAgent
from langchain.prompts import StringPromptTemplate
from langchain.schema import LLMResult, AgentAction, AgentFinish
from langchain.llms.base import LLM
from langchain.tools import BaseTool
from typing import List, Dict, Any, Optional, Union
import os
import requests
import json
import re

# 1. 定义商品搜索工具
class ProductSearchTool(BaseTool):
    name: str = "Product Search"
    description: str = "搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
            {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
            {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
        ]
        results = [p for p in product_db if query.lower() in p["name"].lower()]
        return results[:3]

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        return self._run(query)

# 2. 自定义DeepSeek LLM类
class DeepSeekLLM(LLM):
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            parsed_text = self._parse_llm_output(text)
            results.append({"text": parsed_text})
        return LLMResult(generations=results)

    def _parse_llm_output(self, text: str) -> str:
        """解析模型输出，提取实际回复内容"""
        # 移除Thought:等前缀，提取实际回复
        if "Thought:" in text:
            thought, action = text.split("Thought:", 1)
            if "Action:" in action:
                _, response = action.split("Action:", 1)
                return response.strip()
        return text.strip()

# 3. 自定义提示词模板类
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # 获取中间步骤（如果有）
        intermediate_steps = kwargs.pop("intermediate_steps", [])
        thoughts = ""

        # 构建中间步骤的思考过程
        for action, observation in intermediate_steps:
            thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\n"

        # 设置工具描述
        tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        tool_names = ", ".join([tool.name for tool in self.tools])

        # 格式化提示词
        return self.template.format(
            tool_descriptions=tool_descriptions,
            tool_names=tool_names,
            thoughts=thoughts,
            **kwargs
        )

    def _get_prompt_dict(self) -> Dict[str, Any]:
        return {"tools": self.tools}

# 4. 自定义输出解析器
class CustomOutputParser:
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        # 检查是否为工具调用
        if "<|FunctionCallBegin|>" in text and "<|FunctionCallEnd|>" in text:
            # 提取函数调用部分
            start_idx = text.index("<|FunctionCallBegin|>") + len("<|FunctionCallBegin|>")
            end_idx = text.index("<|FunctionCallEnd|>")

            try:
                function_call = json.loads(text[start_idx:end_idx].strip())
                if isinstance(function_call, list) and len(function_call) > 0:
                    call = function_call[0]
                    return AgentAction(
                        tool=call["name"],
                        tool_input=call["parameters"],
                        log=f"Thought: {text.split('<|FunctionCallBegin|>')[0].strip()}\nAction: {call['name']}\nAction Input: {json.dumps(call['parameters'])}",
                    )
            except json.JSONDecodeError:
                pass

        # 如果不是工具调用，直接返回最终回答
        return AgentFinish(
            return_values={"output": text.strip()},
            log=text,
        )

# 5. 初始化工具和模型
load_dotenv()

tools = [ProductSearchTool()]

llm = DeepSeekLLM(
    model_name="deepseek-ai/deepseek-chat",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 6. 配置并初始化Agent
prompt = CustomPromptTemplate(
    template="""
你是商品推荐助手，必须遵循以下规则：
1. 所有推荐必须先调用Product Search工具
2. 工具调用格式：<|FunctionCallBegin|>[{"name":"Product Search","parameters":{"query":"需求描述"}}]<|FunctionCallEnd|>
3. 若工具返回结果，需按格式推荐并说明理由
4. 若无结果，需询问用户更详细的需求

可用工具:
{tool_descriptions}

问题: {input}
{thoughts}
思考: """,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

agent = ZeroShotAgent(
    llm_chain=llm,
    tools=tools,
    prompt=prompt,
    output_parser=output_parser,
    verbose=True
)

agent_executor = initialize_agent(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.run(user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()

AttributeError: 'ProductSearchTool' object has no attribute 'get'

In [43]:
from dotenv import load_dotenv
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor
from langchain.prompts import StringPromptTemplate
from langchain.schema import LLMResult, AgentAction, AgentFinish
from langchain.llms.base import LLM
from langchain.tools import BaseTool
from typing import List, Dict, Any, Optional, Union, Tuple
import os
import requests
import json
import re
import warnings

# 忽略警告
warnings.filterwarnings("ignore")

# 1. 定义商品搜索工具
class ProductSearchTool(BaseTool):
    name: str = "Product Search"
    description: str = "搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"

    def _run(self, query: str) -> List[Dict[str, Any]]:
        product_db = [
            {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
            {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
            {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
        ]
        results = [p for p in product_db if query.lower() in p["name"].lower()]
        return results[:3]

    async def _arun(self, query: str) -> List[Dict[str, Any]]:
        return self._run(query)

# 2. 自定义DeepSeek LLM类
class DeepSeekLLM(LLM):
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            results.append({"text": text})
        return LLMResult(generations=results)

# 3. 自定义提示词模板类
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # 获取中间步骤（如果有）
        intermediate_steps = kwargs.pop("intermediate_steps", [])
        thoughts = ""

        # 构建中间步骤的思考过程
        for action, observation in intermediate_steps:
            thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\n"

        # 设置工具描述
        tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        tool_names = ", ".join([tool.name for tool in self.tools])

        # 格式化提示词
        return self.template.format(
            tool_descriptions=tool_descriptions,
            tool_names=tool_names,
            thoughts=thoughts,
            **kwargs
        )

# 4. 自定义输出解析器
class CustomOutputParser:
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        # 检查是否为工具调用
        if "Action:" in text and "Action Input:" in text:
            try:
                # 提取工具名称
                action_start = text.index("Action:") + len("Action:")
                action_end = text.index("Action Input:")
                tool = text[action_start:action_end].strip()

                # 提取工具输入
                input_start = action_end + len("Action Input:")
                input_text = text[input_start:].strip()

                # 尝试解析为JSON，如果失败则直接使用文本
                try:
                    tool_input = json.loads(input_text)
                except json.JSONDecodeError:
                    tool_input = {"query": input_text}

                return AgentAction(
                    tool=tool,
                    tool_input=tool_input,
                    log=text
                )
            except Exception as e:
                # 解析失败，作为最终回答
                return AgentFinish(
                    return_values={"output": text},
                    log=text
                )

        # 如果不是工具调用，直接返回最终回答
        return AgentFinish(
            return_values={"output": text},
            log=text
        )

# 5. 初始化工具和模型
load_dotenv()

tools = [ProductSearchTool()]

llm = DeepSeekLLM(
    model_name="deepseek-ai/deepseek-chat",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 6. 配置并初始化Agent
prompt = CustomPromptTemplate(
    template="""
你是商品推荐助手，必须遵循以下规则：
1. 所有推荐必须先调用Product Search工具
2. 工具调用格式：
Action: Product Search
Action Input: 查询内容
3. 若工具返回结果，需按格式推荐并说明理由
4. 若无结果，需询问用户更详细的需求

可用工具:
{tool_descriptions}

问题: {input}
{thoughts}
思考: """,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

# 创建LLMChain
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 初始化Agent
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    output_parser=output_parser,
    verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.run(input=user_input)
        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()

AttributeError: 'ProductSearchTool' object has no attribute 'get'

In [44]:
from dotenv import load_dotenv
from langchain.agents import AgentType, AgentExecutor, ZeroShotAgent
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.llms.base import LLM
from langchain.tools import tool
import os
import requests
import json
import re
import warnings

# 忽略警告
warnings.filterwarnings("ignore")

# 1. 使用装饰器定义工具（推荐方式）
@tool
def product_search(query: str) -> str:
    """搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"""
    product_db = [
        {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
        {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
        {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
    ]
    results = [p for p in product_db if query.lower() in p["name"].lower()]

    # 格式化结果为JSON字符串
    if results:
        return json.dumps(results[:3], ensure_ascii=False)
    return "没有找到匹配的商品，请提供更详细的需求"

# 2. 自定义DeepSeek LLM类
class DeepSeekLLM(LLM):
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            results.append({"text": text})
        return LLMResult(generations=results)

# 3. 自定义提示词模板类
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: list

    def format(self, **kwargs) -> str:
        # 获取中间步骤（如果有）
        intermediate_steps = kwargs.pop("intermediate_steps", [])
        thoughts = ""

        # 构建中间步骤的思考过程
        for action, observation in intermediate_steps:
            try:
                # 解析工具返回的JSON结果
                obs_data = json.loads(observation)
                if isinstance(obs_data, list) and len(obs_data) > 0:
                    thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: 找到了{len(obs_data)}个匹配商品\n"
                else:
                    thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\n"
            except:
                thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\n"

        # 设置工具描述
        tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

        # 格式化提示词
        return self.template.format(
            tool_descriptions=tool_descriptions,
            thoughts=thoughts,
            **kwargs
        )

# 4. 自定义输出解析器
class CustomOutputParser:
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        # 检查是否为工具调用
        if "Action:" in text and "Action Input:" in text:
            try:
                # 提取工具名称
                action_start = text.index("Action:") + len("Action:")
                action_end = text.index("Action Input:")
                tool = text[action_start:action_end].strip()

                # 提取工具输入
                input_start = action_end + len("Action Input:")
                tool_input = text[input_start:].strip()

                return AgentAction(
                    tool=tool,
                    tool_input=tool_input,
                    log=text
                )
            except Exception as e:
                # 解析失败，作为最终回答
                return AgentFinish(
                    return_values={"output": text},
                    log=text
                )

        # 如果不是工具调用，直接返回最终回答
        return AgentFinish(
            return_values={"output": text},
            log=text
        )

# 5. 初始化工具和模型
load_dotenv()

# 使用装饰器创建的工具会自动包含必要的属性
tools = [product_search]

llm = DeepSeekLLM(
    model_name="deepseek-ai/deepseek-chat",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 6. 配置并初始化Agent
prompt = CustomPromptTemplate(
    template="""
你是商品推荐助手，必须遵循以下规则：
1. 所有推荐必须先调用Product Search工具
2. 工具调用格式：
Action: Product Search
Action Input: 查询内容
3. 若工具返回结果，需按格式推荐并说明理由
4. 若无结果，需询问用户更详细的需求

可用工具:
{tool_descriptions}

问题: {input}
{thoughts}
思考: """,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

# 创建LLMChain
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 初始化Agent
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    output_parser=output_parser,
    verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.run(input=user_input)

        # 尝试解析推荐结果
        try:
            if isinstance(response, str) and response.startswith('[{"id"'):
                products = json.loads(response)
                if products:
                    print("商品推荐助手: 为您推荐以下商品：")
                    for i, p in enumerate(products, 1):
                        print(f"{i}. {p['name']} - ¥{p['price']}")
                        print(f"   品牌: {p['brand']}, 特点: {p['features']}")
                    continue
        except:
            pass

        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()


ValidationError: 1 validation error for ZeroShotAgent
output_parser
  Input should be a valid dictionary or instance of AgentOutputParser [type=model_type, input_value=<__main__.CustomOutputPar...t at 0x000002BA53E16BA0>, input_type=CustomOutputParser]
    For further information visit https://errors.pydantic.dev/2.11/v/model_type

In [45]:
from dotenv import load_dotenv
from langchain.agents import AgentType, AgentExecutor, ZeroShotAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.llms.base import LLM
from langchain.tools import tool
import os
import requests
import json
import re
import warnings
from typing import Union, List, Dict

# 忽略警告
warnings.filterwarnings("ignore")

# 1. 使用装饰器定义工具
@tool
def product_search(query: str) -> str:
    """搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"""
    product_db = [
        {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
        {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
        {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
    ]
    results = [p for p in product_db if query.lower() in p["name"].lower()]

    # 格式化结果为JSON字符串
    if results:
        return json.dumps(results[:3], ensure_ascii=False)
    return "没有找到匹配的商品，请提供更详细的需求"

# 2. 自定义DeepSeek LLM类
class DeepSeekLLM(LLM):
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            results.append({"text": text})
        return LLMResult(generations=results)

# 3. 自定义提示词模板类
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: list

    def format(self, **kwargs) -> str:
        # 获取中间步骤（如果有）
        intermediate_steps = kwargs.pop("intermediate_steps", [])
        thoughts = ""

        # 构建中间步骤的思考过程
        for action, observation in intermediate_steps:
            try:
                # 解析工具返回的JSON结果
                obs_data = json.loads(observation)
                if isinstance(obs_data, list) and len(obs_data) > 0:
                    thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: 找到了{len(obs_data)}个匹配商品\n"
                else:
                    thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\n"
            except:
                thoughts += f"\nAction: {action.tool}\nAction Input: {action.tool_input}\nObservation: {observation}\n"

        # 设置工具描述
        tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

        # 格式化提示词
        return self.template.format(
            tool_descriptions=tool_descriptions,
            thoughts=thoughts,
            **kwargs
        )

# 4. 自定义输出解析器（继承AgentOutputParser）
class CustomOutputParser(AgentOutputParser):
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        # 检查是否为工具调用
        if "Action:" in text and "Action Input:" in text:
            try:
                # 提取工具名称
                action_start = text.index("Action:") + len("Action:")
                action_end = text.index("Action Input:")
                tool = text[action_start:action_end].strip()

                # 提取工具输入
                input_start = action_end + len("Action Input:")
                tool_input = text[input_start:].strip()

                return AgentAction(
                    tool=tool,
                    tool_input=tool_input,
                    log=text
                )
            except Exception as e:
                # 解析失败，作为最终回答
                return AgentFinish(
                    return_values={"output": text},
                    log=text
                )

        # 如果不是工具调用，直接返回最终回答
        return AgentFinish(
            return_values={"output": text},
            log=text
        )

# 5. 初始化工具和模型
load_dotenv()

tools = [product_search]

llm = DeepSeekLLM(
    model_name="deepseek-ai/deepseek-chat",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 6. 配置并初始化Agent
prompt = CustomPromptTemplate(
    template="""
你是商品推荐助手，必须遵循以下规则：
1. 所有推荐必须先调用Product Search工具
2. 工具调用格式：
Action: Product Search
Action Input: 查询内容
3. 若工具返回结果，需按格式推荐并说明理由
4. 若无结果，需询问用户更详细的需求

可用工具:
{tool_descriptions}

问题: {input}
{thoughts}
思考: """,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

# 创建LLMChain
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 初始化Agent
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    tools=tools,
    output_parser=output_parser,
    verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.run(input=user_input)

        # 尝试解析推荐结果
        try:
            if isinstance(response, str) and response.startswith('[{"id"'):
                products = json.loads(response)
                if products:
                    print("商品推荐助手: 为您推荐以下商品：")
                    for i, p in enumerate(products, 1):
                        print(f"{i}. {p['name']} - ¥{p['price']}")
                        print(f"   品牌: {p['brand']}, 特点: {p['features']}")
                    continue
        except:
            pass

        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()


`agent_scratchpad` should be a variable in prompt.input_variables. Did not find it, so adding it at the end.


ValidationError: 1 validation error for ZeroShotAgent
  Value error, Got unexpected prompt type <class '__main__.CustomPromptTemplate'> [type=value_error, input_value={'llm_chain': LLMChain(ve...rser(), 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [47]:
from dotenv import load_dotenv
from langchain.agents import AgentType, AgentExecutor, ZeroShotAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish, LLMResult
from langchain.llms.base import LLM
from langchain.tools import tool
import os
import requests
import json
import re
import warnings
from typing import Union, List, Dict, Optional, Any

# 忽略警告
warnings.filterwarnings("ignore")

# 1. 使用装饰器定义工具
@tool
def product_search(query: str) -> str:
    """搜索商品数据库，获取匹配的商品信息。输入应为商品名称或描述，如'蓝牙耳机'或'智能手表'"""
    product_db = [
        {"id": 1, "name": "无线蓝牙耳机", "price": 299, "category": "电子产品", "brand": "SoundMax", "features": "降噪功能"},
        {"id": 2, "name": "运动蓝牙耳机", "price": 399, "category": "电子产品", "brand": "SportSound", "features": "防水设计"},
        {"id": 3, "name": "智能手表", "price": 899, "category": "电子产品", "brand": "SmartGiant", "features": "健康监测"},
    ]
    results = [p for p in product_db if query.lower() in p["name"].lower()]

    # 格式化结果为JSON字符串
    if results:
        return json.dumps(results[:3], ensure_ascii=False)
    return "没有找到匹配的商品，请提供更详细的需求"

# 2. 自定义DeepSeek LLM类
class DeepSeekLLM(LLM):
    model_name: str = "deepseek-ai/deepseek-chat"
    temperature: float = 0.2
    siliconflow_api_key: str
    siliconflow_api_base: str = "https://api.siliconflow.cn/v1"

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.siliconflow_api_key}"
        }
        payload = {
            "model": self.model_name,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": self.temperature,
            "max_tokens": 2048
        }
        response = requests.post(
            f"{self.siliconflow_api_base}/chat/completions",
            headers=headers,
            data=json.dumps(payload)
        )
        if response.status_code != 200:
            raise Exception(f"API调用失败: {response.text}")
        return response.json()["choices"][0]["message"]["content"]

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        results = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            results.append({"text": text})
        return LLMResult(generations=results)

# 3. 自定义提示词模板类（使用agent_scratchpad变量）
class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: list

    def format(self, **kwargs) -> str:
        # 获取中间步骤（agent_scratchpad）
        agent_scratchpad = kwargs.pop("agent_scratchpad", "")

        # 构建中间步骤的思考过程
        thoughts = ""
        if agent_scratchpad:
            thoughts = agent_scratchpad

        # 设置工具描述
        tool_descriptions = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        tool_names = ", ".join([tool.name for tool in self.tools])

        # 格式化提示词
        return self.template.format(
            tool_descriptions=tool_descriptions,
            tool_names=tool_names,
            agent_scratchpad=thoughts,
            **kwargs
        )

    @property
    def _prompt_type(self) -> str:
        return "zero-shot-react-description"

# 4. 自定义输出解析器
class CustomOutputParser(AgentOutputParser):
    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        # 检查是否为工具调用
        if "Action:" in text and "Action Input:" in text:
            try:
                # 提取工具名称
                action_start = text.index("Action:") + len("Action:")
                action_end = text.index("Action Input:")
                tool = text[action_start:action_end].strip()

                # 提取工具输入
                input_start = action_end + len("Action Input:")
                tool_input = text[input_start:].strip()

                return AgentAction(
                    tool=tool,
                    tool_input=tool_input,
                    log=text
                )
            except Exception as e:
                # 解析失败，作为最终回答
                return AgentFinish(
                    return_values={"output": text},
                    log=text
                )

        # 如果不是工具调用，直接返回最终回答
        return AgentFinish(
            return_values={"output": text},
            log=text
        )

# 5. 初始化工具和模型
load_dotenv()

tools = [product_search]

llm = DeepSeekLLM(
    model_name="deepseek-ai/DeepSeek-R1",
    temperature=0.2,
    siliconflow_api_key=os.environ.get("SILICONFLOW_API_KEY"),
    siliconflow_api_base=os.environ.get("SILICONFLOW_API_BASE", "https://api.siliconflow.cn/v1")
)

# 6. 配置并初始化Agent
prompt = CustomPromptTemplate(
    template="""
你是商品推荐助手，必须遵循以下规则：
1. 所有推荐必须先调用Product Search工具
2. 工具调用格式：
Action: Product Search
Action Input: 查询内容
3. 若工具返回结果，需按格式推荐并说明理由
4. 若无结果，需询问用户更详细的需求

可用工具:
{tool_descriptions}

问题: {input}
{agent_scratchpad}
思考: """,
    tools=tools,
    input_variables=["input", "agent_scratchpad"]
)

output_parser = CustomOutputParser()

# 初始化Agent
agent = ZeroShotAgent(
    llm_chain=LLMChain(llm=llm, prompt=prompt),
    tools=tools,
    output_parser=output_parser,
    verbose=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# 7. 测试对话
def run_conversation():
    print("商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)")
    while True:
        user_input = input("用户: ")
        if user_input.lower() == '退出':
            print("商品推荐助手: 感谢使用，再见！")
            break

        response = agent_executor.run(input=user_input)

        # 尝试解析推荐结果
        try:
            if isinstance(response, str) and response.startswith('[{"id"'):
                products = json.loads(response)
                if products:
                    print("商品推荐助手: 为您推荐以下商品：")
                    for i, p in enumerate(products, 1):
                        print(f"{i}. {p['name']} - ¥{p['price']}")
                        print(f"   品牌: {p['brand']}, 特点: {p['features']}")
                    continue
        except:
            pass

        print(f"商品推荐助手: {response}")

if __name__ == "__main__":
    if not os.environ.get("SILICONFLOW_API_KEY"):
        print("请设置SILICONFLOW_API_KEY环境变量")
    else:
        run_conversation()

商品推荐助手: 您好！请问您需要什么商品？(输入'退出'结束对话)


> Entering new AgentExecutor chain...


ValidationError: 1 validation error for LLMResult
generations.0
  Input should be a valid list [type=list_type, input_value={'text': '\n根据您的...一步优化推荐！'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type